In [5]:
import pandas as pd  # Para manejar los datos en DataFrames
from sklearn.model_selection import train_test_split  # Para dividir datos en entrenamiento/prueba
from sklearn.preprocessing import StandardScaler  # Para normalizar los datos
from sklearn.ensemble import RandomForestClassifier  # Modelo de clasificación
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix  # Métricas de evaluación

file_path = r"C:\Users\Emiliano\Desktop\SS python VSC\DatosFelkipeVadillo\Metadatos Eugenia V1.xlsx"
xls = pd.ExcelFile(file_path)  # Se lee el archivo de Excel
df = pd.read_excel(xls, sheet_name='cohorte_princesa_corregidaDESC_')

#  Filtrar solo los casos de interés: Term, Preterm, Postterm (excluyendo Abortion)
df_filtered = df[df['desenlace_parto'].isin(['Term', 'Preterm', 'Postterm'])].copy()

#  Reasignar etiquetas a la variable objetivo (conversión a números)
df_filtered['desenlace_parto'] = df_filtered['desenlace_parto'].replace({
    'Term': 0,      # 0 → Parto a término
    'Preterm': 1,   # 1 → Parto pretérmino
    'Postterm': 2   # 2 → Parto postérmino
})

#  Seleccionar solo características maternas: edad, peso pregestacional y talla
columns_maternal = ['edad_cronologicamujer', 'peso_pregestacional_kg', 'talla_mujer_cm']
df_maternal = df_filtered[columns_maternal + ['desenlace_parto']]

#  Manejo de valores faltantes (rellenar con la media de cada columna)
df_maternal = df_maternal.fillna(df_maternal.mean())

#  Separar características (X) y la variable objetivo (y)
X_maternal = df_maternal.drop(columns=['desenlace_parto'])  # Variables predictoras
y_maternal = df_maternal['desenlace_parto']  # Variable objetivo (clase a predecir)

#  Normalizar características para mejorar el rendimiento del modelo
scaler = StandardScaler()
X_maternal_scaled = scaler.fit_transform(X_maternal)  # Se escalan los datos

#  Dividir los datos en conjunto de entrenamiento (80%) y prueba (20%)
X_train_m, X_test_m, y_train_m, y_test_m = train_test_split(
    X_maternal_scaled, y_maternal, test_size=0.2, random_state=42, stratify=y_maternal
)

# 📌 Entrenar un modelo Random Forest con balanceo de clases
model_maternal = RandomForestClassifier(n_estimators=100, random_state=42, class_weight="balanced")
model_maternal.fit(X_train_m, y_train_m)  # Se entrena el modelo con los datos de entrenamiento

#  Realizar predicciones en el conjunto de prueba
y_pred_maternal = model_maternal.predict(X_test_m)

#  Evaluar el modelo con métricas de desempeño
accuracy_maternal = accuracy_score(y_test_m, y_pred_maternal)  # Precisión del modelo
classification_rep_maternal = classification_report(y_test_m, y_pred_maternal)  # Reporte de clasificación
conf_matrix_maternal = confusion_matrix(y_test_m, y_pred_maternal)  # Matriz de confusión

#  Mostrar resultados
print(f"Precisión del modelo: {accuracy_maternal:.2f}")  # Se imprime la precisión del modelo
print("Reporte de clasificación:\n", classification_rep_maternal)  # Se imprime el reporte detallado
print("Matriz de confusión:\n", conf_matrix_maternal)  # Se imprime la matriz de confusión



Precisión del modelo: 0.88
Reporte de clasificación:
               precision    recall  f1-score   support

           0       0.86      1.00      0.92        18
           1       1.00      0.40      0.57         5
           2       1.00      1.00      1.00         1

    accuracy                           0.88        24
   macro avg       0.95      0.80      0.83        24
weighted avg       0.89      0.88      0.85        24

Matriz de confusión:
 [[18  0  0]
 [ 3  2  0]
 [ 0  0  1]]


C:\Users\Emiliano\AppData\Local\Temp\ipykernel_17284\983426855.py:15: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_filtered['desenlace_parto'] = df_filtered['desenlace_parto'].replace({


Indica que el modelo tiene un 88% de precisión en el conjunto de prueba.
2. Reporte de clasificación
Contiene las métricas clave para cada clase (Term, Preterm, Postterm):
Precision: Qué porcentaje de las predicciones de cada clase fueron correctas.
Recall: Qué porcentaje de los casos reales de cada clase fueron correctamente identificados.
F1-score: Combinación de Precision y Recall.
Support: Cantidad de casos en cada clase.
3. Matriz de confusión
Muestra cómo se distribuyen las predicciones entre las clases reales.
Por ejemplo, [[47 2 1] significa que:
47 casos de "Term" fueron correctamente clasificados como "Term".
2 casos de "Term" fueron mal clasificados como "Preterm".
1 caso de "Term" fue mal clasificado como "Postterm".